# 02 — Bayesian Hierarchical Model (Posterior Probability of Loss)
Significance: partial pooling, credible intervals, probability-of-loss for prioritization.


In [ ]:
import pandas as pd
df = pd.read_csv('../data/service_orders_synthetic.csv', parse_dates=['order_dt'])
df['week_id'] = df['order_dt'].dt.isocalendar().week.astype(int)
df.groupby('region')['margin_pct'].agg(['mean','count']).reset_index()

## PyMC Model Skeleton (run after installing PyMC/ArviZ)

In [ ]:
import pymc as pm, arviz as az, numpy as np
y = df[['region','margin_pct']].dropna(); regions = y['region'].astype('category'); idx = regions.cat.codes.values; y_obs = y['margin_pct'].values
with pm.Model() as model:
    mu_overall = pm.Normal('mu_overall', mu=0.15, sigma=0.1)
    sigma_group = pm.HalfNormal('sigma_group', sigma=0.1)
    sigma = pm.HalfNormal('sigma', sigma=0.1)
    mu_group = pm.Normal('mu_group', mu=mu_overall, sigma=sigma_group, shape=regions.cat.categories.size)
    pm.Normal('y_like', mu=mu_group[idx], sigma=sigma, observed=y_obs)
    idata = pm.sample(1000, tune=1000, target_accept=0.9, chains=2, cores=1, progressbar=False)
az.summary(idata, var_names=['mu_overall','sigma_group','sigma','mu_group'])

In [ ]:
post = az.extract(idata, var_names=['mu_group']).to_numpy()
import pandas as pd
probs = (post < 0).mean(axis=0)
pd.DataFrame({'region': regions.cat.categories, 'p_loss': probs}).sort_values('p_loss', ascending=False)